# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в заголовке которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

In [2]:
KEYWORDS = ['python', 'парсинг']
params = {"q":KEYWORDS,"target_type":"posts","order":"relevance"} 
req = requests.get("https://habr.com/ru/search/", params=params) # Читинг
# Я занес в параметры поиска искомые ключевые слова, 
# т.к. в сырой выдаче я их не находил. А в поиске по Хабру выполняется условие по наличию в заголовках
soup = BeautifulSoup(req.text)
posts = soup.find_all('article', 'tm-articles-list__item')
links=[]
for post in posts:
    heads = post.find_all('a', class_="tm-article-snippet__title-link")
    for head in heads:
        links.append('https://habr.com'+head.get('href')) # Создаем список ссылок. 

habr_search = pd.DataFrame()        
for link in links:
    soup = BeautifulSoup(requests.get(link).text)
    time.sleep(0.3)
    date = pd.to_datetime(soup.find('span', class_="tm-article-snippet__datetime-published").find('time').get('datetime'))
    title = soup.find('h1', class_="tm-article-snippet__title tm-article-snippet__title_h1").text
    text = soup.find('div', class_="tm-article-body").text.replace('\n','').replace('\r','')
    row = {'Date':date,'Title':title,'Link':link,'Text':text}
    habr_search = pd.concat([habr_search, pd.DataFrame([row])]) 


In [3]:
habr_search = habr_search.reset_index().drop(['index'], axis=1)
habr_search.head()

,Date,Title,Link,Text
0,2021-06-17 05:49:15+00:00,Вышла библиотека PyWhat для автоматического па...,https://habr.com/ru/company/dcmiran/news/t/563...,"Разработана удобная библиотека PyWhat, котора..."
1,2011-06-14 15:24:25+00:00,Парсинг на Pуthon. Как собрать архив Голубятен,https://habr.com/ru/post/121815/,Статья описывает разработку скрипта на языке ...
2,2020-10-01 14:17:40+00:00,Как выбрать решение для парсинга сайтов: класс...,https://habr.com/ru/post/521646/,Парсинг или как его еще иногда называют web s...
3,2020-07-31 07:00:46+00:00,Как проанализировать рынок фотостудий с помощь...,https://habr.com/ru/post/512208/,В предыдущей статье в рамках коммерческого пр...
4,2015-03-06 14:40:20+00:00,Веб-парсинг на Ruby,https://habr.com/ru/post/252379/,"Это перевод статьи «Web Scraping with Ruby», ..."


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только заголовки статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы. Данные (почтовые адреса) передаются на сервис в json-формате (параметр `json` функции `post`).

In [126]:
EMAIL = {"emailAddresses":["user1@yandex.ru"]}
URL = "https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"
HEADERS = {
    'Vaar-Header-App-Build-Version':'1.0.0',
    'Vaar-Header-App-Product-Name':'hackcheck-web-avast',
    'Vaar-Version':'0'
}
req = requests.post(url=URL, headers=HEADERS, json=EMAIL)


    
        
    

In [128]:
reb = req.json()
reb['breaches']
leakage = pd.DataFrame()
for values in reb.values():
    for key, value in values.items():
        site = value['site']
        date = value['publishDate']
        description = value['description']
        row = {'Date':date,'Site':site,'Description':description}
        leakage = pd.concat([leakage, pd.DataFrame([row])])
    break
leakage

,Date,Site,Description
0,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
0,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
0,2020-11-05T00:00:00Z,vimeworld.ru,"In January 2018, the Russian Minecraft server ..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
